#### IMPORTS

In [21]:
import os

def find_r_installation():
    possible_paths = []

    # Add standard paths for Windows
    if os.getenv('ProgramFiles'):
        possible_paths.append(os.path.join(os.getenv('ProgramFiles'), 'R'))
    if os.getenv('ProgramFiles(x86)'):
        possible_paths.append(os.path.join(os.getenv('ProgramFiles(x86)'), 'R'))
    if os.getenv('LOCALAPPDATA'):
        possible_paths.append(os.path.join(os.getenv('LOCALAPPDATA'), 'Programs', 'R'))
    
    # Add standard path for Unix-like systems
    if os.getenv('HOME'):
        possible_paths.append(os.path.join(os.getenv('HOME'), 'R'))

    for path in possible_paths:
        if os.path.exists(path):
            for root, dirs, files in os.walk(path):
                if 'bin' in dirs:
                    if 'R.exe' in os.listdir(os.path.join(root, 'bin')):  # Check specifically for Windows
                        return os.path.join(root)
                    if 'R' in os.listdir(os.path.join(root, 'bin')):  # Check for Unix-like systems
                        return os.path.join(root)

    return None

r_path = find_r_installation()
if r_path:
    print(f"R is installed at: {r_path}")
else:
    print("R installation not found.")

os.environ['R_HOME'] = r_path

R is installed at: C:\Users\maceo.valente\AppData\Local\Programs\R\R-4.3.2


In [22]:
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import r

In [29]:

# Charger le fichier R contenant la fonction
r.source("Rfunctions.R")

# Définir la fonction R
f_gen_usm = ro.globalenv['generate_usm']
f_gen_ini = ro.globalenv['generate_ini']
f_gen_tec = ro.globalenv['generate_tec']
f_gen_sols = ro.globalenv['generate_sols']
f_gen_sta = ro.globalenv['generate_sta']
f_settings = ro.globalenv['settings']
f_change_scenar = ro.globalenv['change_scenar']


# # Fonction Python pour appeler la fonction R
# def call_r_function(param1, param2, param3):
#     result = my_r_function(param1, param2, param3)
#     return result[0]

# # Exemple d'utilisation
# params = [1, 2, 3]
# result = call_r_function(*params)
# print(f"Result from R function: {result}")


In [31]:
f_settings("C:/Users/maceo.valente/Documents/Automatisation/TEST_MVA/TSAgriPV/R_related")

Setting working directory to: C:/Users/maceo.valente/Documents/Automatisation/TEST_MVA/TSAgriPV/R_related 
Get working dir :  C:/Users/maceo.valente/Documents/Automatisation/TEST_MVA/TSAgriPV/R_related 
Javastics path :  C:/Users/maceo.valente/Documents/Automatisation/TEST_MVA/TSAgriPV/JavaSTICS/JavaStics.exe 
Workspace path :  C:/Users/maceo.valente/Documents/Automatisation/TEST_MVA/TSAgriPV/R_related/Rproject 
Le dossier existe déjà: Rproject/weatherFiles/ 


<rpy2.rinterface_lib.sexp.NULLType object at 0x0000019AADBF8A48> [RTYPES.NILSXP]